Todo:


- obrácené vnoření A a B v 1998ps_stenprot_003schuz_s003023.htm
- "(pokračuje" v 1920ns_se_182schuz_s182005
- jiné formáty v histortm
- procedurální věci v 1945pns_stenprot_003schuz_s003001.h- Federální shromáždění ČSFR, 1992 – 1992 
Sněmovna lid nemá ani tučné, ani odkazyustů

In [4]:
import os
import csv
import json
import random
from bs4 import BeautifulSoup
import concurrent.futures

In [5]:
random.sample(os.listdir("downloads/schuze"), 10)

['2021ps_stenprot_088schuz_s088223.htm',
 '2006ps_stenprot_027schuz_s027138.htm',
 '2017ps_stenprot_007schuz_s007239.htm',
 '2017ps_stenprot_041schuz_s041169.htm',
 '1998ps_stenprot_007schuz_s007016.htm',
 '2006ps_stenprot_052schuz_s052056.htm',
 '1848urrs_stenprot_075schuz_s075003.htm',
 '2010ps_stenprot_038schuz_s038032.htm',
 '1998ps_stenprot_036schuz_s036314.htm',
 '1990snr_stenprot_023schuz_s023040.htm']

In [6]:
vsechny = list(set([x[0:10] for x in os.listdir("downloads/schuze")]))
vsechny.sort()
print(vsechny)

['1848urrs_s', '1861skc_st', '1918ns_ps_', '1920ns_ps_', '1920ns_se_', '1925ns_ps_', '1925ns_se_', '1929ns_ps_', '1929ns_se_', '1935ns_ps_', '1935ns_se_', '1939ssk_st', '1939ssr_st', '1944snr_st', '1945pns_st', '1946snr_st', '1946uns_st', '1948ns_ste', '1948snr_st', '1954ns_ste', '1954snr_st', '1960ns_ste', '1960snr_st', '1964ns_ste', '1964snr_st', '1968cnr_st', '1969cnr_st', '1969fs_sl_', '1969fs_sls', '1969fs_sn_', '1971cnr_st', '1971fs_sl_', '1971fs_sls', '1971fs_sn_', '1976cnr_st', '1976fs_sl_', '1976fs_sls', '1976fs_sn_', '1981cnr_st', '1981fs_sl_', '1981fs_sls', '1981fs_sn_', '1981snr_st', '1986cnr_st', '1986fs_sl_', '1986fs_sls', '1986fs_sn_', '1990cnr_st', '1990fs_sl_', '1990fs_sls', '1990fs_sn_', '1990snr_st', '1992cnr_st', '1992fs_sl_', '1992fs_sls', '1992fs_sn_', '1993ps_ste', '1996ps_pss', '1996ps_ste', '1998nr_ste', '1998ps_ste', '2002ps_pss', '2002ps_ste', '2004se_ste', '2006ps_pss', '2006ps_ste', '2006se_ste', '2008se_ste', '2010ps_pss', '2010ps_ste', '2013ps_ste', '2017

In [7]:
kam = "data_raw/schuze"

try:
    os.makedirs(kam)
except:
    pass

In [8]:
import re

In [9]:
# re.search(r"\d{1,2}\. \w{4,12} [12]\d{3}", "zxz").group()

In [10]:
def oscrapovani_stenozaznamu(soubor, zdroj=False):
    
    def ziskej_metadata(soubor):
        if "psse" not in soubor:
            komora_kratce = soubor.split("_")[0][4:]
        else:
            komora_kratce = "psse"
        obdobi = soubor.split("_")[0][0:4]
        schuze = soubor.split("_")[-1][1:4]
        cast = soubor.split("_")[-1][-7:-4]
        # print(komora_kratce)
        # print(obdobi)
        # print(schuze)
        # print(cast)
        return komora_kratce, int(obdobi), int(schuze), int(cast.removesuffix("."))

    def najdi_datum(polivka):
        mesice = {
            'ledna':1,
            'února':2,
            'března':3,
            'dubna':4,
            'května':5,
            'června':6,
            'července':7,
            'srpna':8,
            'září':9,
            'října':10,
            'listopadu':11,
            'prosince':12
        }

        surove_datum = ""
        if polivka.find(class_="date"):
            surove_datum = polivka.find(class_="date").text.replace('\xa0',' ').lower()
        if surove_datum == "":
            try:
                surove_datum = re.search(r"\d{1,2}\. \w{4,12} [12]\d{3}", polivka.text).group()
            except:
                surove_datum = polivka.find("a", href="../index.htm").text.lower()

        try:
            rok = int(surove_datum.split(" ")[-1])
            mesic = surove_datum.split(" ")[-2]        
            den = int(surove_datum.split(" ")[-3].removesuffix('.'))
            return f"{rok}-{mesice[mesic]:02}-{den:02}"
        except:
            return None

    def najdi_komoru_a_schuzi(polivka):
        odkazy = polivka.find_all('a')
        komora, schuze = None, None
        try:
            komora = soup.find("h1").text.strip()
        except:
            komora = soup.find(id="breadcrumb").find_all("a")[1].text.strip()
        finally:
            pass
        for a in odkazy:
            if 'href' in a.attrs:
                if './index.htm' in a['href']:
                    schuze = a.text.strip()
        return komora, schuze

    def ciste_odstavce(seznam):
        return ' '.join(seznam).replace('\xa0',' ').replace('***','').removeprefix(':').strip()
    
    metadata = ziskej_metadata(soubor)

    slovnik_metadata = {
        'komora': metadata[0],
        'obdobi': metadata[1],
        'schuze': metadata[2],
        'cast': metadata[3],
        'soubor': soubor
    }

    if "1996ps" in soubor:
        parsovatko = "html5lib" # kvůli 1996ps_stenprot_016schuz_s016005.htm aj., odkazy začínají mimo odstavce
    else:
        parsovatko = "lxml"
    
    with open(os.path.join("downloads/schuze/",soubor), "r", encoding="windows-1250") as schuze:
        schuze = schuze.read().replace('\n',' ')
        soup = BeautifulSoup(schuze, parsovatko)

    slovnik_metadata['datum'] = najdi_datum(soup)
    slovnik_metadata['komora_komplet'], slovnik_metadata['schuze_komplet'] = najdi_komoru_a_schuzi(soup)
    slovnik_metadata['prepsano'] = True
    slovnik_metadata['autorizovano'] = True

    orez = 130
    
    projevy = []
    
    if "nebyl dosud přepsán. Prosím, zkuste později." in soup.text:
        slovnik_metadata['prepsano'] = False
        return slovnik_metadata, projevy
        
    if ("Neprošlo jazykovou korekturou, neautorizováno" in soup.text) or ("Neprošlo opravou po digitalizaci" in soup.text):
        slovnik_metadata['autorizovano'] = False
    
    uz_byl_tucnej = False
    poradi = 1
    projev = {'mluvci':None,'poradi':poradi,'soubor':soubor,'mluvci_id':None}
    odstavce = []

    vsechny_odstavce = soup.find_all("p")
        
    for p in vsechny_odstavce[5:-10]:
        if zdroj == True:
            print("____" + p.text[0:orez])

        if ("1992fs_sl" in soubor) or ("1990fs_sl" in soubor):

            if (':' in p.text.strip()[0:orez - 30]) and (len([l for l in p.text.strip().split(":")[0] if l.isupper()]) > 1):
                
                projev['text'] = ciste_odstavce(odstavce)
                if projev['text'] != '':
                    poradi += 1
                    projevy.append(projev)
                projev = {}
                odstavce = []
                uz_byl_tucnej = True
                projev['poradi'] = poradi
                projev['soubor'] = soubor
                projev['mluvci'] = p.text.split(':')[0].strip()
                projev['mluvci_id'] = None
                odstavce.append(' '.join(p.text.split(':')[1:]).strip())

            else:
                odstavce.append(p.text.strip())

        else:

            if zdroj:
                if p.find("b"):
                    print("TUČNÉ")

            if ':' in p.text.strip()[0:orez] and (
                    p.find('b', string=lambda text: (text.strip() in p.text.strip()[0:orez]) and (len(text.strip()) >= 3) if text else False) 
                    or 
                    p.find('u', string=lambda text: (text.strip() in p.text.strip()[0:orez]) and (len(text.strip()) >= 3) if text else False)
                ) and (
                    "výsledkem:" not in p.text # kvůli 2006se_stenprot_015schuz_s015018.htm
                ):

                pocitadlo_zvyrazneni = 0
                prvni_tucne_je_druhe_tucne = False

                if len(p.find_all("b")) > 1: # kvůli 1920ns_se_stenprot_195schuz_s195001.htm
                    if zdroj == True:
                        print("počítám tučnost")
                    for tucne in p.find_all("b"):
                        if tucne.text.strip() in p.text.strip()[0:orez]:
                            pocitadlo_zvyrazneni += 1
                    if p.find_all("b")[0].text.strip() == p.find_all("b")[1].text.strip(): # kvůli 1996ps_stenprot_021schuz_s021021.htm
                        prvni_tucne_je_druhe_tucne = True
                if (pocitadlo_zvyrazneni > 1) & (prvni_tucne_je_druhe_tucne == False):
                    if zdroj == True:
                        print("dvakrát")
                    odstavce.append(p.text.strip())

                else:
                    if zdroj == True:
                        print('mluvčí (: + a/b)')

                    if ("resoluční návrh" in p.text.lower()) or ("pozměňovací návrh" in p.text.lower()) or ("doplňovací návrh" in p.text.lower()):
                        odstavce.append(p.text.strip())
                        # kvůli 1920ns_ps_stenprot_078schuz_s078006.htm

                    else:
                        
                        projev['text'] = ciste_odstavce(odstavce)
                        if projev['text'] != '':
                            poradi += 1
                            projevy.append(projev)
                        projev = {}
                        odstavce = []
                        projev['poradi'] = poradi
                        projev['soubor'] = soubor
                        projev['mluvci'] = p.text.split(':')[0].strip()
                        try:
                            projev['mluvci_id'] = odstavec.find("a")['href'].split('?')[-1]
                        except:
                            projev['mluvci_id'] = None
                        odstavce.append(p.text.replace(projev['mluvci'],'').strip())

            elif p.find("b") and (len(p.text.strip()) < 100) and (p.text.strip().endswith(':')):
                projev['text'] = ciste_odstavce(odstavce)
                if projev['text'] != '':
                    poradi += 1
                    projevy.append(projev)
                projev = {}
                odstavce = []
                projev['poradi'] = poradi
                projev['soubor'] = soubor
                projev['mluvci'] = p.find('b').text.replace(':','').strip()
    
            elif p.find('a'): ## častá chyba, viz testy: je odkaz, není dvojtečka. kód se tu zbytečně duplikuje, whatever
                if zdroj == True:
                    print('ODKAZ')
                if p.text.strip().startswith(p.find('a').text.strip()) and (p.find("a").get("id") != "_d") and(p.find("a").get("name") != "OLE_LINK1"):
                    # p.find("a").get("name") != "OLE_LINK1" kvůli 1946uns_stenprot_087schuz_s087005.htm
                    
                    if zdroj == True:
                        print('mluvčí p.find(a)')
    
                    projev['text'] = ciste_odstavce(odstavce)
                    if projev['text'] != '':
                        poradi += 1
                        projevy.append(projev)
                    projev = {}
                    odstavce = []
                    projev['poradi'] = poradi
                    projev['soubor'] = soubor
                    projev['mluvci'] = p.find("a").text.strip()
                    try:
                        projev['mluvci_id'] = odstavec.find("a")['href'].split('?')[-1]
                    except:
                        projev['mluvci_id'] = None
                    odstavce.append(p.text.replace(projev['mluvci'],'').strip())

                else:
                    odstavce.append(p.text.strip())
                    
            else:
                odstavce.append(p.text.strip())
        
    projev['text'] = ciste_odstavce(odstavce)
    projevy.append(projev)    
    
    return slovnik_metadata, projevy

In [11]:
def html2data(soubor):
    
    ciste = oscrapovani_stenozaznamu(soubor)
    soubor = soubor.replace('.htm','')
    
    with open(os.path.join(kam, f'prepis_{soubor}.ndjson'), "w+", encoding="utf-8") as export:
        for projev in ciste[1]:
            export.write(json.dumps(projev) + '\n')

    with open(os.path.join(kam, f'meta_{soubor}.csv'), "w+", newline="", encoding="utf-8") as export_file:
        fieldnames = ciste[0].keys()
        writer = csv.DictWriter(export_file, fieldnames=fieldnames, quoting=csv.QUOTE_STRINGS)
        writer.writeheader()
        writer.writerow(ciste[0])

In [12]:
oscrapovani_stenozaznamu("1996ps_stenprot_016schuz_s016005.htm", zdroj=True)

____
____ 
____        Místopředseda PSP Jan Kasal: Děkuji panu poslanci Ivanu Maškovi a v tuto chvíli zbývá rozhodnout o proceduře, která tady b
TUČNÉ
počítám tučnost
mluvčí (: + a/b)
____ Dámy a pánové, v hlasování pořadové číslo 4 rozhodneme o tom, že nejprve budeme hlasovat o doprovodných usneseních.
ODKAZ
____ 
____ Kdo souhlasí s tímto návrhem, ať zvedne ruku a stiskne tlačítko. Kdo je proti? Děkuji.
____ Hlasování skončilo výsledkem 153 pro, 3 byli proti, takže návrh byl přijat.
____ 
____ Zřejmě můžeme dát hlasovat o doprovodném usnesení, které přednesla paní poslankyně Ivana Plechatá. Je tomu tak, pane zpravodaji?
____ 
____        Poslanec Jozef Wagner: Ano, obě stanoviska jsou doporučující.
TUČNÉ
počítám tučnost
mluvčí (: + a/b)
____ 
____        Místopředseda PSP Jan Kasal: Děkuji.
TUČNÉ
počítám tučnost
mluvčí (: + a/b)
____ 
____ Zahájil jsem hlasování pořadové číslo 5. Táži se, kdo je pro? Kdo je proti? Děkuji.
ODKAZ
____ Toto hlasování skončilo. Pro se vyslovilo 154 posl

({'komora': 'ps',
  'obdobi': 1996,
  'schuze': 16,
  'cast': 5,
  'soubor': '1996ps_stenprot_016schuz_s016005.htm',
  'datum': '1997-11-13',
  'komora_komplet': 'Parlament České republiky, Poslanecká sněmovna 1996 – 1998',
  'schuze_komplet': '16. schůze',
  'prepsano': True,
  'autorizovano': True},
 [{'poradi': 1,
   'soubor': '1996ps_stenprot_016schuz_s016005.htm',
   'mluvci': 'Místopředseda PSP Jan Kasal',
   'mluvci_id': None,
   'text': 'Děkuji panu poslanci Ivanu Maškovi a v tuto chvíli zbývá rozhodnout o proceduře, která tady byla naznačena, protože se zdá, že pan poslanec Exner nehodlá požádat sněmovnu, aby mohl své návrhy vzít zpět. Je tomu tak. Takže myslím, že můžeme postupovat tak, jak navrhl pan zpravodaj. Přesto o tom dám hlasovat, aby z toho nevznikl nějaký nebezpečný precedens. Dámy a pánové, v hlasování pořadové číslo 4 rozhodneme o tom, že nejprve budeme hlasovat o doprovodných usneseních.  Kdo souhlasí s tímto návrhem, ať zvedne ruku a stiskne tlačítko. Kdo je pro

In [13]:
oscrapovani_stenozaznamu("1993ps_stenprot_016schuz_s016005.htm", zdroj=True)

____
____ Předseda PSP Milan Uhde: Děkuji panu ministru práce a sociálních věcí Jindřichu Vodičkovi za výklad, který přednesl, udílím slovo
TUČNÉ
počítám tučnost
mluvčí (: + a/b)
____ Poslanec Jaroslav Štrait: Vážený pane předsedo, dámy a pánové, rozpočtový výbor a výbor pro sociální politiku a zdravotnictví pro
TUČNÉ
počítám tučnost
mluvčí (: + a/b)
____ Doporučuji po rozpravě Poslanecké sněmovně tento vládní návrh zákona ve znění společné zprávy přijmout.  
____ Předseda PSP Milan Uhde: Děkuji panu společnému zpravodaji a otevírám k předloženému vládnímu návrhu rozpravu. Nemám písemné přih
TUČNÉ
počítám tučnost
mluvčí (: + a/b)
____ Poslankyně Eva Fischerová: Pane předsedo, dámy a pánové, já, i když vím, že to je velmi pravděpodobně volání bez odezvy, tak přes
TUČNÉ
počítám tučnost
mluvčí (: + a/b)
____ Jsem přesvědčena, že zákon o předčasném poskytování starobního důchodu v této formě, jak je předkládán, poškozuje ve většině příp
____ Jsem přesvědčena, že by bylo správné odložit při

({'komora': 'ps',
  'obdobi': 1993,
  'schuze': 16,
  'cast': 5,
  'soubor': '1993ps_stenprot_016schuz_s016005.htm',
  'datum': '1994-02-15',
  'komora_komplet': 'Parlament České republiky, Poslanecká sněmovna 1993 – 1996',
  'schuze_komplet': '16. schůze',
  'prepsano': True,
  'autorizovano': True},
 [{'poradi': 1,
   'soubor': '1993ps_stenprot_016schuz_s016005.htm',
   'mluvci': 'Předseda PSP Milan Uhde',
   'mluvci_id': None,
   'text': 'Děkuji panu ministru práce a sociálních věcí Jindřichu Vodičkovi za výklad, který přednesl, udílím slovo společnému zpravodaji výborů panu poslanci Jaroslavu Štraitovi a prosím ho, aby odůvodnil společnou zprávu výborů. Prosím, pane společný zpravodaji.'},
  {'poradi': 2,
   'soubor': '1993ps_stenprot_016schuz_s016005.htm',
   'mluvci': 'Poslanec Jaroslav Štrait',
   'mluvci_id': None,
   'text': 'Vážený pane předsedo, dámy a pánové, rozpočtový výbor a výbor pro sociální politiku a zdravotnictví projednaly na svých schůzích v listopadu 1993 a v led

In [65]:
oscrapovani_stenozaznamu("2013ps_stenprot_049schuz_s049999.htm", zdroj=False)

({'komora': 'ps',
  'obdobi': 2013,
  'schuze': 49,
  'cast': 999,
  'soubor': '2013ps_stenprot_049schuz_s049999.htm',
  'datum': None,
  'komora_komplet': 'Parlament České republiky, Poslanecká sněmovna 2013 – 2017',
  'schuze_komplet': '49. schůze',
  'prepsano': True,
  'autorizovano': False},
 [{'poradi': 1,
   'soubor': '2013ps_stenprot_049schuz_s049999.htm',
   'mluvci': 'Dolů',
   'mluvci_id': None,
   'text': 'Prostě Evropa by měla být garantem bezpečí pro občany. To jsou - řekněme - ty věcné cíle, se kterými konec konců také jako předseda české vlády pojedu do Bratislavy.'},
  {'poradi': 2,
   'soubor': '2013ps_stenprot_049schuz_s049999.htm',
   'mluvci': 'Místopředsedkyně PSP Jaroslava Pokorná Jermanová',
   'mluvci_id': None,
   'text': 'Děkuji. Paní poslankyně Fischerová, chcete položit doplňující otázku? Prosím.'},
  {'poradi': 3,
   'soubor': '2013ps_stenprot_049schuz_s049999.htm',
   'mluvci': 'Poslankyně Jana Fischerová',
   'mluvci_id': None,
   'text': 'Děkuji za odpo

In [123]:
oscrapovani_stenozaznamu("2010ps_stenprot_054schuz_s054069.htm", zdroj=True)

____
____ 
TUČNÉ
____(Jednání pokračovalo v 11.02 hodin.)
____ 
____Předsedkyně PSP Miroslava Němcová  Vážené paní poslankyně, vážení páni poslanci, budeme pokračovat v našem dnešním jednání. Chci v
TUČNÉ
mluvčí p.find(a)
____ 
____Budeme moci zahájit projednávání bodu číslo 111. Je to 
____ 
____111.  Vládní návrh, kterým se předkládá Parlamentu České republiky  k vyslovení souhlasu s ratifikací Protokol proti pašování přis
TUČNÉ
____Prosím, aby se slova ujal ministr vnitra Jan Kubice. Prosím, pane ministře. (Slovo požaduje poslanec Filip pro faktickou poznámku.
____Pardon, pan kolega - faktická poznámka může být vznesena pouze v průběhu rozpravy, a rozprava se nekoná. (Poslanec Filip uplatňuje
____ 
____Poslanec Vojtěch Filip: Děkuji, paní předsedkyně. Já jsem rád, že tady máme pana ministra vnitra, ale nemyslím si, že je možné se 
TUČNÉ
mluvčí (: + a/b)
____ 
____Předsedkyně PSP Miroslava Němcová  Děkuji. Máme tedy dvě možnosti. Já se musím zeptat pana ministra, zda je ochoten teď t

({'komora': 'ps',
  'obdobi': 2010,
  'schuze': 54,
  'cast': 69,
  'soubor': '2010ps_stenprot_054schuz_s054069.htm',
  'datum': '2013-06-13',
  'komora_komplet': 'Parlament České republiky, Poslanecká sněmovna 2010 – 2013',
  'schuze_komplet': '54. schůze',
  'prepsano': True,
  'autorizovano': True},
 [{'mluvci': None,
   'poradi': 1,
   'soubor': '2010ps_stenprot_054schuz_s054069.htm',
   'mluvci_id': None,
   'text': '(Jednání pokračovalo v 11.02 hodin.)'},
  {'poradi': 2,
   'soubor': '2010ps_stenprot_054schuz_s054069.htm',
   'mluvci': 'Předsedkyně PSP Miroslava Němcová',
   'mluvci_id': None,
   'text': 'Vážené paní poslankyně, vážení páni poslanci, budeme pokračovat v našem dnešním jednání. Chci vás seznámit s omluvami, které mi byly ještě doručeny. Obdržela jsem omluvu pana ministra financí Miroslava Kalouska, který se omlouvá z jednání dnes odpoledne. Pak tady mám upozornění, že se omlouvá také pan ministr Blažek. Ano, pan ministr Blažek. To byly omluvy, které mezi tím byly d

In [85]:
oscrapovani_stenozaznamu("1920ns_se_stenprot_195schuz_s195001.htm", zdroj=True)


Schůze zahájena v 17 hodin 15 minut.
Přítomni:
Předseda Donát.
TUČNÉ
Místopředseda: Kadlčák, Klofáč, Niessner, dr Soukup.
TUČNÉ
mluvčí (: + a/b)
Zapisovatelé: dr Krouský, Svěcený.
TUČNÉ
mluvčí (: + a/b)
107 senátorů podle presenční listiny. 
Zástupci vlády: ministři: dr Markovič, Srba, Udržal; za ministerstvo vnitra odborový přednosta dr Bobek.
TUČNÉ
mluvčí (: + a/b)
Z kanceláře senátní: senátní tajemník dr Šafařovič; jeho zástupci dr Bartoušek, dr Trmal.
TUČNÉ
mluvčí (: + a/b)
Předseda (zvoní): Zahajuji schůzi. 
TUČNÉ
mluvčí (: + a/b)
Sděluji, že udělil jsem dovolenou na dobu 8 dnů panu sen. Fahrnerovi; pro dnešní schůzi panu sen. Lisému, Cholkovi a Hec
TUČNÉ
Navrhuji, aby byla udělena zdravotní dovolená na dobu 4 neděl panu sen. dr Kaperniczkymu, na dobu 4 měsíců panu sen. Dul
TUČNÉ
Kdo s tímto návrhem souhlasí, nechť zvedne ruku. (Děje se.)
To je většina. Žádané dovolené se udělují.
TUČNÉ
Senátní tajemník dr Šafařovič (čte):
TUČNÉ
mluvčí (: + a/b)
Tiskem rozdáno:
Tisk 1822. Naléhav

({'komora': 'ns',
  'obdobi': 1920,
  'schuze': 195,
  'cast': 1,
  'soubor': '1920ns_se_stenprot_195schuz_s195001.htm',
  'datum': '1924-03-18',
  'komora_komplet': 'NS RČS 1920-1925',
  'schuze_komplet': '195. schůze',
  'prepsano': True,
  'autorizovano': True},
 [{'mluvci': None,
   'poradi': 1,
   'soubor': '1920ns_se_stenprot_195schuz_s195001.htm',
   'mluvci_id': None,
   'text': 'Schůze zahájena v 17 hodin 15 minut. Přítomni: Předseda Donát.'},
  {'poradi': 2,
   'soubor': '1920ns_se_stenprot_195schuz_s195001.htm',
   'mluvci': 'Místopředseda',
   'mluvci_id': None,
   'text': 'Kadlčák, Klofáč, Niessner, dr Soukup.'},
  {'poradi': 3,
   'soubor': '1920ns_se_stenprot_195schuz_s195001.htm',
   'mluvci': 'Zapisovatelé',
   'mluvci_id': None,
   'text': 'dr Krouský, Svěcený. 107 senátorů podle presenční listiny.'},
  {'poradi': 4,
   'soubor': '1920ns_se_stenprot_195schuz_s195001.htm',
   'mluvci': 'Zástupci vlády',
   'mluvci_id': None,
   'text': 'ministři: dr Markovič, Srba, Udr

In [19]:
def process_file(soubor):
    try:
        html2data(soubor)
    except Exception as e:
        print(f"{soubor}: {e}")

In [15]:
# soubory = [x for x in os.listdir("downloads/schuze") if ("1992fs_sl" in x) or ("1990fs_sl" in x)]
# soubory = os.listdir("downloads/schuze")
# pocitadlo = 0

# with concurrent.futures.ThreadPoolExecutor() as executor:
#    futures = [executor.submit(process_file, soubor) for soubor in soubory]

#    for future in concurrent.futures.as_completed(futures):
#        pocitadlo += 1
#        if pocitadlo % 1000 == 0:
#            print(f"{pocitadlo}/{len(soubory)}")

In [ ]:
directory = "downloads/schuze"
files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
# files = [f for f in files if "199" in f]
# files = [f for f in files if ('1992fs_sl' in f) or ('1990fs_sl' in f)] 
total_files = len(files)

batch_size = 8

# Process files in batches to reduce memory usage and improve progress tracking
for i in range(0, total_files, batch_size):
    batch = files[i:i + batch_size]
    
    # Use ProcessPoolExecutor for CPU-bound tasks or ThreadPoolExecutor for I/O-bound tasks
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Map is more memory efficient than list comprehension for large batches
        futures = {executor.submit(process_file, f): f for f in batch}
        
        for j, future in enumerate(concurrent.futures.as_completed(futures), 1):
            try:
                future.result()  # Get the result to raise any exceptions
            except Exception as e:
                print(f"Error processing file {futures[future]}: {e}")
            
            # Update progress every batch_size files
            if (i + j) % batch_size == 0:
                print(f"{i + j}/{total_files}")